[View in Colaboratory](https://colab.research.google.com/github/Keshav-Aggarwal/Beard-Face-Dataset/blob/master/Working%20Custom%20Dataset%20Model.ipynb)

In [1]:
# !wget -O master.zip https://github.com/Keshav-Aggarwal/Beard-Face-Dataset/raw/master/Beard%20Dataset.zip
# !unzip master.zip
# !ls -ltr

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import glob
import tensorflow.contrib.slim as slim
# %matplotlib.pyplot inline

in_image=tf.placeholder(tf.float32,[None,None,None,4])
gt_image=tf.placeholder(tf.float32,[None,None,None,4])

def upsample_and_concat(x1, x2, output_channels, in_channels):

    pool_size = 2
    deconv_filter = tf.Variable(tf.truncated_normal( [pool_size, pool_size, output_channels, in_channels], stddev=0.02))
    deconv = tf.nn.conv2d_transpose(x1, deconv_filter, tf.shape(x2) , strides=[1, pool_size, pool_size, 1])

    deconv_output =  tf.concat([deconv, x2],3)
    deconv_output.set_shape([None, None, None, output_channels*2])

    return deconv_output



def lrelu(x):
    return tf.maximum(x*0.2,x)
  
regularizer_l2 = tf.contrib.layers.l2_regularizer(scale=0.1)

conv1=slim.conv2d(in_image,32,[3,3], rate=1, activation_fn=lrelu,scope='g_conv1_1',weights_regularizer = regularizer_l2)
conv1=slim.conv2d(conv1,32,[3,3], rate=1, activation_fn=lrelu,scope='g_conv1_2',weights_regularizer = regularizer_l2)
pool1=slim.max_pool2d(conv1, [2, 2], padding='SAME' )

conv2=slim.conv2d(pool1,64,[3,3], rate=1, activation_fn=lrelu,scope='g_conv2_1',weights_regularizer = regularizer_l2)
conv2=slim.conv2d(conv2,64,[3,3], rate=1, activation_fn=lrelu,scope='g_conv2_2',weights_regularizer = regularizer_l2)
pool2=slim.max_pool2d(conv2, [2, 2], padding='SAME' )

conv3=slim.conv2d(pool2,128,[3,3], rate=1, activation_fn=lrelu,scope='g_conv3_1',weights_regularizer = regularizer_l2)
conv3=slim.conv2d(conv3,128,[3,3], rate=1, activation_fn=lrelu,scope='g_conv3_2',weights_regularizer = regularizer_l2)
pool3=slim.max_pool2d(conv3, [2, 2], padding='SAME' )

conv4=slim.conv2d(pool3,256,[3,3], rate=1, activation_fn=lrelu,scope='g_conv4_1',weights_regularizer = regularizer_l2)
conv4=slim.conv2d(conv4,256,[3,3], rate=1, activation_fn=lrelu,scope='g_conv4_2',weights_regularizer = regularizer_l2)
pool4=slim.max_pool2d(conv4, [2, 2], padding='SAME' )

conv5=slim.conv2d(pool4,512,[3,3], rate=1, activation_fn=lrelu,scope='g_conv5_1',weights_regularizer = regularizer_l2)
conv5=slim.conv2d(conv5,512,[3,3], rate=1, activation_fn=lrelu,scope='g_conv5_2',weights_regularizer = regularizer_l2)

up6 =  upsample_and_concat( conv5, conv4, 256, 512  )
conv6=slim.conv2d(up6,  256,[3,3], rate=1, activation_fn=lrelu,scope='g_conv6_1',weights_regularizer = regularizer_l2)
conv6=slim.conv2d(conv6,256,[3,3], rate=1, activation_fn=lrelu,scope='g_conv6_2',weights_regularizer = regularizer_l2)

up7 =  upsample_and_concat( conv6, conv3, 128, 256  )
conv7=slim.conv2d(up7,  128,[3,3], rate=1, activation_fn=lrelu,scope='g_conv7_1',weights_regularizer = regularizer_l2)
conv7=slim.conv2d(conv7,128,[3,3], rate=1, activation_fn=lrelu,scope='g_conv7_2',weights_regularizer = regularizer_l2)

up8 =  upsample_and_concat( conv7, conv2, 64, 128 )
conv8=slim.conv2d(up8,  64,[3,3], rate=1, activation_fn=lrelu,scope='g_conv8_1',weights_regularizer = regularizer_l2)
conv8=slim.conv2d(conv8,64,[3,3], rate=1, activation_fn=lrelu,scope='g_conv8_2',weights_regularizer = regularizer_l2)

up9 =  upsample_and_concat( conv8, conv1, 32, 64 )
conv9=slim.conv2d(up9,  32,[3,3], rate=1, activation_fn=lrelu,scope='g_conv9_1',weights_regularizer = regularizer_l2)
conv9=slim.conv2d(conv9,32,[3,3], rate=1, activation_fn=lrelu,scope='g_conv9_2',weights_regularizer = regularizer_l2)

conv10=slim.conv2d(conv9,4,[1,1], rate=1, activation_fn=None, scope='g_conv10',weights_regularizer = regularizer_l2)
# out = tf.depth_to_space(conv10,4)



out_image=conv10
sess = tf.Session()


In [3]:

beard = glob.glob('beard/*.jpg')
nobeard = glob.glob('no_beard/*.jpg')
imgs_beard = []
imgs_no_beard = []

for path_b, path_n in zip(beard, nobeard):
  imgs_beard.append(plt.imread(path_b))
  imgs_no_beard.append(plt.imread(path_n))

In [0]:
reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
reg_constant = 0.1  # Choose an appropriate one.

G_loss = tf.reduce_mean(tf.abs(out_image - gt_image)) + reg_constant * sum(reg_losses)

G_opt=tf.train.AdamOptimizer(learning_rate=0.0005).minimize(G_loss)
sess.run(tf.global_variables_initializer())
for i in range(250):
  _,G_current,output=sess.run([G_opt, G_loss, out_image],feed_dict={in_image:imgs_beard,gt_image:imgs_no_beard})
  print("Loss for epoch {v_one} is {v_two}".format(v_one = i, v_two = G_current))


Loss for epoch 0 is 170.28529357910156
Loss for epoch 1 is 158.93052673339844
Loss for epoch 2 is 148.85037231445312
Loss for epoch 3 is 137.88595581054688
Loss for epoch 4 is 126.87071228027344
Loss for epoch 5 is 112.57830047607422
Loss for epoch 6 is 90.16329193115234
Loss for epoch 7 is 84.96222686767578
Loss for epoch 8 is 71.4530029296875
Loss for epoch 9 is 69.40386199951172
Loss for epoch 10 is 69.098388671875
Loss for epoch 11 is 68.7762222290039
Loss for epoch 12 is 68.16761016845703
Loss for epoch 13 is 67.38208770751953
Loss for epoch 14 is 66.58126831054688
Loss for epoch 15 is 65.8241195678711
Loss for epoch 16 is 65.11715698242188
Loss for epoch 17 is 64.39945983886719
Loss for epoch 18 is 63.64033508300781
Loss for epoch 19 is 62.86319351196289


In [0]:
plt.imshow(sess.run(output[0]))

TypeError: ignored

In [0]:
plt.imshow(output[0]/255)

ValueError: ignored